# Price Providers

In [44]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import date

In [45]:
from simple_back.price_providers import YahooFinanceProvider, TimeLeakError

In [46]:
price = YahooFinanceProvider()

In [47]:
price.clear_cache()

### Getting a symbol

In [48]:
price['AAPL']

,open,close,high,low
1980-12-12,0.405683,0.405683,0.407447,0.405683
1980-12-15,0.386281,0.384517,0.386281,0.384517
1980-12-16,0.358060,0.356296,0.358060,0.356296
1980-12-17,0.365115,0.365115,0.366879,0.365115
1980-12-18,0.375698,0.375698,0.377462,0.375698
...,...,...,...,...
2020-05-22,315.769989,318.890015,319.230011,315.350006
2020-05-26,323.500000,316.730011,324.239990,316.500000
2020-05-27,316.140015,318.109985,318.709991,313.089996
2020-05-28,316.769989,318.250000,323.440002,315.630005


### Getting values at a specific day

In [49]:
price['AAPL','2015-1-2']

open     101.829
close    99.9459
high     101.875
low      98.1358
Name: 2015-01-02 00:00:00, dtype: object

In [50]:
price['AAPL','2020-5-18']

open     313.17
close    314.96
high      316.5
low      310.32
Name: 2020-05-18 00:00:00, dtype: object

### Using a date range

In [51]:
price['AAPL','2015-1-2':'2015-1-7']

,open,close,high,low
2015-01-02,101.829067,99.945885,101.874778,98.135831
2015-01-05,98.995143,97.130241,99.324244,96.362344
2015-01-06,97.395385,97.139420,98.208994,95.649322
2015-01-07,97.998723,98.501518,98.912891,97.541640


### Using relativedelta

In [52]:
price['AAPL',-relativedelta(months=1):].head()

,open,close,high,low
2020-04-30,289.177206,293.006836,293.734876,287.571567
2020-05-01,285.477218,288.289612,298.192797,285.078304
2020-05-04,288.389342,292.368561,292.897129,285.547030
2020-05-05,294.263433,296.756683,300.187399,293.665046
2020-05-06,299.648835,299.818390,302.421329,298.063132


### Using day ints
When a negative integer is used, this is interpreted as the number of days to go back.

In [37]:
price['AAPL',-7:]

,open,close,high,low
2020-05-26,323.500000,316.730011,324.239990,316.500000
2020-05-27,316.140015,318.109985,318.709991,313.089996
2020-05-28,316.769989,318.250000,323.440002,315.630005
2020-05-29,319.250000,317.940002,321.149994,316.470001


Relativedeltas and integers work respective of the end date specified after `:` - If none is specified, the current date is used.

In [38]:
price['AAPL',-7:'2015-1-2']

,open,close,high,low
2014-12-26,102.478164,104.205940,104.690448,102.395893
2014-12-29,104.023095,104.132797,104.918975,103.940816
2014-12-30,103.885969,102.862099,104.141934,102.487294
2014-12-31,103.136355,100.905785,103.419745,100.750378
2015-01-02,101.829067,99.945885,101.874778,98.135831


In [39]:
price['AAPL',-relativedelta(days=7):'2015-1-2']

,open,close,high,low
2014-12-26,102.478164,104.205940,104.690448,102.395893
2014-12-29,104.023095,104.132797,104.918975,103.940816
2014-12-30,103.885969,102.862099,104.141934,102.487294
2014-12-31,103.136355,100.905785,103.419745,100.750378
2015-01-02,101.829067,99.945885,101.874778,98.135831


## Time Leak Protection
During backtesting, the internal states ``current_event`` and ``current_date`` are set internally (you do not have to set these states) and trying to access future values will result in a ``TimeLeakError``.

In [40]:
# setting internal states for demonstration in this notebook, this happens automatically during a backtest
price.current_event = 'open'
price.current_date = date(2020,5,18)

In [41]:
try:
    price['AAPL','2020-5-22']
except TimeLeakError as e:
    print(e)

(datetime.date(2020, 5, 18), Timestamp('2020-05-22 00:00:00'), '2020-05-22 00:00:00 is more recent than 2020-05-18, resulting in time leak')


When accessing multiple values, future events are set to ``None``.

In [42]:
price['AAPL','2020-5-18']

open     313.17
close      None
high       None
low        None
Name: 2020-05-18 00:00:00, dtype: object

In [43]:
price['AAPL',-5:'2020-5-18']

,open,close,high,low
2020-05-13,312.149994,307.649994,315.950012,303.209991
2020-05-14,304.510010,309.540009,309.790009,301.529999
2020-05-15,300.350006,307.709991,307.899994,300.209991
2020-05-18,313.170013,NaN,NaN,NaN
